In [23]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [25]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR ctau-BDT-based (trained on [5,100]GeV): run on all signal mass points [5, 100] GeV

In [26]:
ct = 1

## Background

In [27]:
prefix = './coffea/skimmed/'
suffix = f'_SR_BDT_ctau-{ct}'

In [28]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [29]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000001
Triboson,0.001933,0.000158,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000002
W+jets,0.002310,0.000173,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000001
Z+jets,0.006645,0.000398,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000001
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_Background_Efficiency'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_ctau-1-based_trained_5to100_Background_Efficiency.csv


In [31]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,"38,039.83","1,882.67","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,559.22,349.76,314.03,312.99,312.95,294.00,2.82
DY,"14,495.72",671.44,671.44,623.25,623.25,623.25,623.25,455.65,143.51
Diboson,"4,424.20",307.91,307.91,227.24,213.36,211.27,210.33,189.35,4.66
Triboson,53.00,4.32,4.32,2.85,2.51,2.51,2.50,2.09,0.05
W+jets,"227,212.60","17,009.81","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",146.07
Z+jets,"142,220.15","8,524.55","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",25.28
Total,"432,113.00","28,959.92","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",322.39


In [32]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_Background_EventCount_Weighted'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_ctau-1-based_trained_5to100_Background_EventCount_Weighted.csv


In [33]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,12311,381,381,160,159,159,158,3,0
Top,49818,4688,4688,3058,2348,2337,2336,2006,41
DY,205,13,13,11,11,11,11,9,1
Diboson,18067,1234,1234,915,860,852,848,767,18
Triboson,2257,170,170,116,106,105,104,92,1
W+jets,697657,48425,48425,37015,35472,35415,35377,29090,412
Z+jets,1281459,67427,67427,52627,50986,50934,50903,44480,197
Total,2061774,122338,122338,93902,89942,89813,89737,76447,670


In [34]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_Background_EventCount_Raw'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_ctau-1-based_trained_5to100_Background_EventCount_Raw.csv


In [35]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 322.39


In [36]:
pd.options.display.float_format = '{:,.2f}'.format

bkg_composition = pd.concat([pd.DataFrame(bkg_cts_df['BDT']).transpose(), pd.DataFrame(bkg_cts_raw_df['BDT']).transpose()], ignore_index=True)

bkg_composition = bkg_composition.rename(index={0: 'Weighted Counts', 1: 'Raw Counts'})
bkg_composition

,QCD,Top,DY,Diboson,Triboson,W+jets,Z+jets,Total
Weighted Counts,0.00,2.82,143.51,4.66,0.05,146.07,25.28,322.39
Raw Counts,0.00,41.00,1.00,18.00,1.00,412.00,197.00,670.00


## Signal

In [37]:
# Signal
sig_histo = util.load(f'{prefix}/signal{suffix}.coffea')[0]

In [38]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,84.0,8.0,1.0,80.0,88.0,0.1,sig_2018_Mchi-84p0_dMchi-8p0_ctau-1
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,77.0,14.0,1.0,70.0,84.0,0.2,sig_2018_Mchi-77p0_dMchi-14p0_ctau-1
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,99.0,18.0,1.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-1
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,94.5,9.0,1.0,90.0,99.0,0.1,sig_2018_Mchi-94p5_dMchi-9p0_ctau-1
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,73.5,7.0,1.0,70.0,77.0,0.1,sig_2018_Mchi-73p5_dMchi-7p0_ctau-1
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,63.0,6.0,1.0,60.0,66.0,0.1,sig_2018_Mchi-63p0_dMchi-6p0_ctau-1
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,60.0,20.0,1.0,50.0,70.0,0.4,sig_2018_Mchi-60p0_dMchi-20p0_ctau-1
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.5,1.0,1.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.2,0.5,1.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-1
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,55.0,10.0,1.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-1


In [39]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'computing BDT score',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT']

### Efficiency

In [40]:
pd.options.display.float_format = '{:,.6f}'.format

In [41]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [42]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [43]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,0.058112,0.011195,0.011195,0.009729,0.009420,0.009420,0.009420,0.009342,0.004760
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,0.049041,0.011230,0.011230,0.009552,0.009201,0.009201,0.009201,0.009076,0.005645
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,0.065041,0.015343,0.015343,0.012976,0.012527,0.012527,0.012527,0.012353,0.007517
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,0.066110,0.013011,0.013011,0.011131,0.010744,0.010743,0.010743,0.010641,0.005586
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,0.051428,0.009301,0.009301,0.007902,0.007672,0.007672,0.007672,0.007581,0.003834
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,0.043559,0.007791,0.007791,0.006622,0.006351,0.006351,0.006351,0.006279,0.003164
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,0.029875,0.007027,0.007027,0.005808,0.005631,0.005631,0.005631,0.005523,0.003634
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000677
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.014782,0.002143,0.002143,0.001894,0.001811,0.001811,0.001811,0.001794,0.000719
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.003694


In [44]:
outdir = './csv/Cutflow/Efficiency/'
outname = f'SR_BDT_ctau-based_trained_5to100_Signal_ctau-{int(ct)}_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_ctau-based_trained_5to100_Signal_ctau-1_samples_Efficiency.csv


### Weighted Event count

In [45]:
pd.options.display.float_format = '{:,.2f}'.format

In [46]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [47]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [48]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,"2,122.81",408.93,408.93,355.40,344.10,344.10,344.10,341.27,173.88
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,90.58,20.74,20.74,17.64,16.99,16.99,16.99,16.76,10.43
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,51.58,12.17,12.17,10.29,9.93,9.93,9.93,9.80,5.96
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,"1,627.11",320.24,320.24,273.97,264.43,264.40,264.40,261.91,137.48
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,"2,928.06",529.56,529.56,449.92,436.79,436.79,436.79,431.62,218.28
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,"4,153.77",742.93,742.93,631.48,605.60,605.60,605.60,598.76,301.71
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,5.68,1.33,1.33,1.10,1.07,1.07,1.07,1.05,0.69
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,349.57,288.31,269.62,268.98,268.98,266.40,128.38
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","3,032.72"
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,42.35,36.16,34.55,34.55,34.55,34.28,22.41


In [49]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = f'SR_BDT_ctau-based_trained_5to100_Signal_ctau-{int(ct)}_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_ctau-based_trained_5to100_Signal_ctau-1_samples_EventCount_Weighted.csv


In [50]:
# without m1 = 30 GeV
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,"2,122.81",408.93,408.93,355.40,344.10,344.10,344.10,341.27,173.88
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,90.58,20.74,20.74,17.64,16.99,16.99,16.99,16.76,10.43
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,51.58,12.17,12.17,10.29,9.93,9.93,9.93,9.80,5.96
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,"1,627.11",320.24,320.24,273.97,264.43,264.40,264.40,261.91,137.48
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,"2,928.06",529.56,529.56,449.92,436.79,436.79,436.79,431.62,218.28
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,"4,153.77",742.93,742.93,631.48,605.60,605.60,605.60,598.76,301.71
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,5.68,1.33,1.33,1.10,1.07,1.07,1.07,1.05,0.69
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,349.57,288.31,269.62,268.98,268.98,266.40,128.38
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","3,032.72"
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,42.35,36.16,34.55,34.55,34.55,34.28,22.41


### Raw count

In [51]:
pd.options.display.float_format = '{:,.0f}'.format

In [52]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [53]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [54]:
points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,12770,2460,2460,2138,2070,2070,2070,2053,1046
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,10607,2429,2429,2066,1990,1990,1990,1963,1221
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,31711,7481,7481,6327,6108,6108,6108,6023,3665
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,48974,9639,9639,8246,7959,7958,7958,7883,4138
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,13602,2460,2460,2090,2029,2029,2029,2005,1014
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,11551,2066,2066,1756,1684,1684,1684,1665,839
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,7768,1827,1827,1510,1464,1464,1464,1436,945
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,542,447,418,417,417,413,199
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,516,456,436,436,436,432,173
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,157,134,128,128,128,127,83


In [55]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = f'SR_BDT_ctau-based_trained_5to100_Signal_ctau-{int(ct)}_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_ctau-based_trained_5to100_Signal_ctau-1_samples_EventCount_Raw.csv


## S/sqrt(B)

In [56]:
nBkg

322.387916156102

In [57]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [58]:
pd.options.display.float_format = '{:,.4f}'.format

In [59]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [60]:
# Missing rows
#df[ (df['m1'] == 60) & (df['delta'] == 0.2) ]

df.loc[len(df.index)] = [0, 60, 0.2, 1]
df.loc[len(df.index)] = [0, 80, 0.2, 1] 

df = df.sort_values(by=['m1']) # sort by m1

In [61]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,128.3809,5.0000,0.2000,1.0000,322.3879
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"3,032.7169",5.0000,0.1000,1.0000,322.3879
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,84.9228,10.0000,0.2000,1.0000,322.3879
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"2,036.9709",10.0000,0.1000,1.0000,322.3879
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"1,680.8538",20.0000,0.1000,1.0000,322.3879
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,75.5786,20.0000,0.2000,1.0000,322.3879
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"4,493.0721",30.0000,0.2000,1.0000,322.3879
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"1,939.0330",30.0000,0.1000,1.0000,322.3879
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,45.4102,40.0000,0.2000,1.0000,322.3879
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"1,097.6365",40.0000,0.1000,1.0000,322.3879


In [62]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,128.3809,322.3879,7.1501
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"3,032.7169",322.3879,168.9050
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,84.9228,322.3879,4.7297
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,"2,036.9709",322.3879,113.4476
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,"1,680.8538",322.3879,93.6139
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,75.5786,322.3879,4.2093
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,"4,493.0721",322.3879,250.2384
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,"1,939.0330",322.3879,107.9931
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,45.4102,322.3879,2.5291
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,"1,097.6365",322.3879,61.1321


In [63]:
outdir = './csv/SR_Yields/'
outname = f'SR_BDT_ctau-{int(ct)}-based_trained_5to100_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_BDT_ctau-1-based_trained_5to100_S_over_B.csv


### ctau = 1mm signal points

In [64]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.1
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0,0.1,1.0,"3,032.7169",322.3879,168.9050
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0,0.1,1.0,"2,036.9709",322.3879,113.4476
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0,0.1,1.0,"1,680.8538",322.3879,93.6139
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0,0.1,1.0,"1,939.0330",322.3879,107.9931
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0,0.1,1.0,"1,097.6365",322.3879,61.1321
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,50.0,0.1,1.0,462.3802,322.3879,25.7519
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,60.0,0.1,1.0,301.7103,322.3879,16.8035
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,70.0,0.1,1.0,218.2813,322.3879,12.1570
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,80.0,0.1,1.0,173.8801,322.3879,9.6841
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,90.0,0.1,1.0,137.4827,322.3879,7.6570


In [65]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.2
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0,0.2,1.0,128.3809,322.3879,7.1501
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0,0.2,1.0,84.9228,322.3879,4.7297
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0,0.2,1.0,75.5786,322.3879,4.2093
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0,0.2,1.0,"4,493.0721",322.3879,250.2384
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0,0.2,1.0,45.4102,322.3879,2.5291
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,50.0,0.2,1.0,22.4076,322.3879,1.2480
21,60.0,0.2,1.0,0.0000,322.3879,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,70.0,0.2,1.0,10.4274,322.3879,0.5807
22,80.0,0.2,1.0,0.0000,322.3879,0.0000
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,90.0,0.2,1.0,5.9610,322.3879,0.3320
